# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder, PolynomialFeatures

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, log_loss

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

import optuna
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Setting Dataframe environment
pd.set_option('display.max_rows',25)
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.3f'%x)

## Data Preparation

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s3e24/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e24/test.csv')
df.sample(3)

In [ ]:
test.sample(3)

In [ ]:
# shapes
print('shape of train is :',df.shape)
print('shape of test is :',test.shape)

In [ ]:
# Removing ID column to reduce extra computation as it provides no information
def remove_id(df):
    return df.drop('id',axis=1,inplace=True)

test_id = test['id']
remove_id(df)
remove_id(test)

# Geneerate statistics for each column
df.describe()

In [ ]:
# unique values in all columns
for i in df.columns:
    print(f'{i:19}','-->',df[i].nunique())

### Data Visualization

In [ ]:
# checking data distribution and spotting if there is any leakage
fig =plt.figure(figsize=(13,13))

j=0
for i in test.columns:
    fig.add_subplot(7,4,j+1)
    sns.kdeplot(df,x=i,color='green')
    sns.kdeplot(test,x=i,color='red',fill=True)
    j += 1
plt.tight_layout()
plt.show()

# serum creatinine seems to be different in density amounts 
# in other columns no major data leakage
# many columnns are right skewed

In [ ]:
# adding 1 and transforming all columns into there log values  (as there are some zeroes)
# to see if we can reduce skewness

fig =plt.figure(figsize=(13,13))

for i in range(len(test.columns)):
    fig.add_subplot(7,4,i+1)
    sns.kdeplot(x=np.log1p(df[f'{test.columns[i]}']),color='green')
    sns.kdeplot(x=np.log1p(df[f'{test.columns[i]}']),color='red',fill=True)
plt.tight_layout()
plt.show()

# log is good for eyesight(left), eyesight(right), systolic, relaxation, fasting blood sugar, 
# triglyceride, HDL, LDL, hemoglobin, serum creatinine,
# AST, ALT, Gtp

In [ ]:
# Distributions of all columns on the basis of smoking status

fig =plt.figure(figsize=(13,13))
j=0
for i in test.columns:
    fig.add_subplot(7,4,j+1)
    sns.kdeplot(df,x=i,hue='smoking',fill=True)
    j += 1
plt.tight_layout()
plt.show()

# we can see some significant observaion in distributions.

In [ ]:
# for i in range(1,7):
#     protein_filter = (df['Urine protein']==i)
#     fig =plt.figure(figsize=(13,30))

#     j=0
#     for i in test.columns:
#         fig.add_subplot(11,2,j+1)
#         sns.histplot(df[protein_filter],x=i,hue='smoking',fill=True,bins=100)
#         j += 1
#     plt.tight_layout()
#     plt.show()

# this plot plots histogram of every feature to get noticable variation but
# But no significant observation appeared 

In [ ]:
# For a closer look

fig =plt.figure(figsize=(13,30))

j=0
for i in test.columns:
    fig.add_subplot(11,2,j+1)
    sns.histplot(df,x=i,hue='smoking',fill=True,bins=100)
    j += 1
plt.tight_layout()
plt.show()

Points I observed in above graph:
most of the smokers have:
* 20 < age < 40
* height > 165
* 70 < weight < 100 
* 85 < waist < 105
* triglyceride > 150
* HDL < 50
* Hemoglobin > 50
* There are other observable factors but not so significant

most of the non smokers have:
* age >= 40
* height <=160
* weight <= 65
* waist <= 82
* triglyceride <= 110
* HDL >=55
* hemoglobin <= 14

In [ ]:
def compare(feature_name, comparison = 'only_less',less_than = 0,greater_than = 0):
    #  smoking condition
    do = (df['smoking'] == 1)
    dont = (df['smoking'] == 0)
    
    if comparison == 'only_less':
        print()
        smokers = df[(df[feature_name] < less_than) & do].shape[0]
        non_smokers = df[(df[feature_name] < less_than) & dont].shape[0]
        print('{0:15} {1:5} {2:15} {3:15} {4:30}'.format(feature_name,smokers + non_smokers,smokers,non_smokers,smokers/non_smokers))
    
    elif comparison == 'only_greater':
        print()
        smokers = df[(df[feature_name] > greater_than) & do].shape[0]
        non_smokers = df[(df[feature_name] > greater_than) & dont].shape[0]
        print('{0:15} {1:5} {2:15} {3:15} {4:30}'.format(feature_name,smokers + non_smokers,smokers,non_smokers,smokers/non_smokers))
    
    else :
        print()
        smokers = df[(df[feature_name] < less_than) & (df[feature_name] > greater_than) & do].shape[0]
        non_smokers = df[(df[feature_name] < less_than) & (df[feature_name] > greater_than) & dont].shape[0]
        print('{0:15} {1:5} {2:15} {3:15} {4:30}'.format(feature_name,smokers + non_smokers,smokers,non_smokers,smokers/non_smokers))
    
        
print('{0:15} {1:15} {2:15} {3:15} {4:15}'.format('feature name','Total','smokers','Non-smokers','smoker to Non-smoker ratio'))
compare('age','both',40,20)
compare('height(cm)','only_greater',0,165)
compare('weight(kg)','both',100,70)
compare('waist(cm)','both',105,85)
compare('triglyceride','only_greater',0,150)
compare('HDL','only_less',50)
compare('hemoglobin','only_greater',0,15)

In [ ]:
# Binning

def create_smoker_bins(df):
    age_filter = (df['age']>20)&(df['age']<40)
    height_filter = (df['height(cm)']>165)
    weight_filter = (df['weight(kg)']>70) & (df['weight(kg)']<100)
    waist_filter = (df['waist(cm)']>85) & (df['waist(cm)']<105)
    triglyceride_filter = (df['triglyceride']>150)
    hdl_filter = (df['HDL']<50)
    hemoglobin_filter = (df['hemoglobin']>15)
    
    
    df['age_bin'] = np.where(age_filter,1,0)
    df['height_bin'] = np.where(height_filter,1,0)
    df['weight_bin'] = np.where(weight_filter,1,0)
    df['waist_bin'] = np.where(waist_filter,1,0)
    df['triglyceride_bin'] = np.where(triglyceride_filter,1,0)
    df['hdl_bin'] = np.where(hdl_filter,1,0)
    df['hemoglobin_bin'] = np.where(hemoglobin_filter,1,0)
    df['all_smoker_bin'] = df['age_bin'] + df['height_bin'] + df['weight_bin'] + df['waist_bin'] + \
                        df['triglyceride_bin'] + df['hdl_bin'] + df['hemoglobin_bin']
    return df
df = create_smoker_bins(df)
df.sample(5)

In [ ]:
# Blue Dominance
non_age_filter = (df['age']>= 40)
non_height_filter = (df['height(cm)'] <= 160)
non_weight_filter = (df['weight(kg)'] <= 65)
non_waist_filter = (df['waist(cm)'] <= 82)
non_triglyceride_filter = (df['triglyceride'] <= 110)
non_hdl_filter = (df['HDL'] >= 55)
non_hemoglobin_filter = (df['hemoglobin'] <= 14)

def show_blue_dominance(feature_name,feature_filter):
    do = (df['smoking'] == 1)
    dont = (df['smoking'] == 0)
    smokers = df[feature_name][feature_filter & do].shape[0]
    non_smokers = df[feature_name][feature_filter & dont].shape[0]
    print()
    print('{0:15} {1:5} {2:15} {3:15} {4:30} '.format(feature_name,smokers + non_smokers,smokers,non_smokers,non_smokers/smokers))
    
    
print('{0:15} {1:15} {2:15} {3:15} {4:15} '.format('feature_name','total','smokers','non_smokers','non_smoker to smoker ratio'))
show_blue_dominance('age',non_age_filter)
show_blue_dominance('height(cm)',non_height_filter)
show_blue_dominance('weight(kg)',non_weight_filter)
show_blue_dominance('waist(cm)',non_waist_filter)
show_blue_dominance('triglyceride',non_triglyceride_filter)
show_blue_dominance('HDL',non_hdl_filter)
show_blue_dominance('hemoglobin',non_hemoglobin_filter)

In [ ]:
# Binning

def create_non_bins(df):
    non_age_filter = (df['age']>= 40)
    non_height_filter = (df['height(cm)'] <= 160)
    non_weight_filter = (df['weight(kg)'] <= 65)
    non_waist_filter = (df['waist(cm)'] <= 82)
    non_triglyceride_filter = (df['triglyceride'] <= 110)
    non_hdl_filter = (df['HDL'] >= 55)
    non_hemoglobin_filter = (df['hemoglobin'] <= 14)
    
    df['non_age'] = np.where(non_age_filter,1,0)
    df['non_height'] = np.where(non_height_filter,1,0)
    df['non_weight'] = np.where(non_weight_filter,1,0)
    df['non_waist'] = np.where(non_waist_filter,1,0)
    df['non_triglyceride'] = np.where(non_triglyceride_filter,1,0)
    df['non_hdl'] = np.where(non_hdl_filter,1,0)
    df['non_hemoglobin'] = np.where(non_hemoglobin_filter,1,0)

    df['all_non_bin'] = df['non_age'] + df['non_height'] + df['non_weight'] + df['non_waist'] + df['non_triglyceride'] + \
                        df['non_hdl'] + df['non_hemoglobin']
    return df

df = create_non_bins(df)
df.sample(3)

In [ ]:
check1 = ['HDL','AST','systolic','serum creatinine']
fig = plt.figure(figsize=(8,8))
for i, col in enumerate(check1):
    df1 = pd.DataFrame(df[col].copy())
    df2 = pd.DataFrame(test[col].copy())
    
    df1['label']='df'
    df2['label']='test'
    
    combined_df = pd.concat([df1,df2])
    
    fig.add_subplot(2,2,i+1)
    sns.boxplot(combined_df,x='label',y=col)
    plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Removing Outlier
print(df[df['HDL']<18].shape[0])                  # only 1 sample
df.drop(df[df['HDL']<18].index[0],inplace=True)

### checking correlation

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),annot=True,fmt='.2f',cmap='coolwarm',linewidths=1)
plt.tight_layout()
plt.show()

Some important correlation appeared in original features as follows:
* weight to waist correlation is 0.83
* systolic to ralaxation correlation is 0.75
* cholesterol to LDL correlation is 0.81 

> Let's try multipy these to create new and better features

In [ ]:
df['weight_waist'] = df['weight(kg)'] * df['waist(cm)']
df['systo_relax'] = df['systolic'] * df['relaxation']
df['chole_ldl'] = df['Cholesterol'] * df['LDL']

Now let's see mutual information of these features

In [ ]:
mut = mutual_info_classif(df.drop('smoking',axis=1),df['smoking'])
mut = pd.Series(mut,index=df.drop('smoking',axis=1).columns)
mut = mut.sort_values(ascending=True)

def plotting(s):
    plt.figure(figsize=(12,15),dpi=100)
    width = np.arange(len(s))
    ticks=s.index
    plt.barh(width,s)
    plt.yticks(width,ticks)
    plt.grid()
    
plotting(mut)

In [ ]:
# Now we'll see correlation between features with MI score > 0.06 and target column
# This is a very important step in feature selection process
# I'll select those features with less mutual correlation but strong correlation with target feature

good_col = ['height(cm)','hemoglobin','all_non_bin','all_smoker_bin','Gtp','non_hemoglobin','hemoglobin_bin',
            'non_height','height_bin','weight(kg)','triglyceride','serum creatinine','weight_waist',
            'non_weight','non_triglyceride','smoking']

plt.figure(figsize=(9,7))
sns.heatmap(df[good_col].corr(),annot=True,fmt='.2f',cmap='coolwarm',linewidths=1)
plt.show()

Now we'll drop some features from this set based on there mutual correlation
* Dropping serum creatinine because of it's uneven distribution in train and test datasets

In [ ]:
# MI score > 0.06
select1 = ['height(cm)','hemoglobin','all_non_bin','Gtp','hemoglobin_bin',
            'non_height','triglyceride','weight_waist',
            'non_weight','non_triglyceride']

# MI score > 0.08
select2 = ['height(cm)','hemoglobin','all_non_bin','Gtp','hemoglobin_bin',
            'non_height','weight_waist']

In [ ]:
test = create_smoker_bins(test)
test = create_non_bins(test)

test['weight_waist'] = test['weight(kg)'] * test['waist(cm)']

# Model Building and Hyper Parameter Tuning

In [ ]:
dff = df[select2 + ['smoking']].copy()
dff.shape, test[select2].shape

# it turns out that select1 is better

## Cross Validation

In [ ]:
x_temp = dff.drop('smoking',axis=1).copy()
y_temp = dff['smoking'].copy()
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=1)

out_cv = {}
out_proba_cv = {}

def cal_cv(model,label,stand = True, proba = True):
    auc_scores = []
    proba_auc_scores = []
    
    for train_idx,val_idx in skf.split(x_temp,y_temp):
        x_tr = x_temp.iloc[train_idx]
        y_tr = y_temp.iloc[train_idx]
        
        x_val = x_temp.iloc[val_idx]
        y_val = y_temp.iloc[val_idx]
        
        if stand == True:
            sc = StandardScaler()
            x_tr = sc.fit_transform(x_tr)
            x_val = sc.transform(x_val)
        
        model.fit(x_tr,y_tr)
        y_pred = model.predict(x_val)
        auc_scores.append(roc_auc_score(y_val,y_pred))
        
        if proba == True:
            y_proba = model.predict_proba(x_val)[:,1]
            proba_auc_scores.append(roc_auc_score(y_val,y_proba))
    
    final_auc = np.mean(auc_scores)
    
    if len(proba_auc_scores) != 0:
        final_proba_auc = np.mean(proba_auc_scores)
    else:
        final_proba_auc = 'NOT PREDICTED'
    
    out_cv[label]= final_auc
    out_proba_cv[label] = final_proba_auc
    
    return label, final_auc, final_proba_auc

Let's see score for Simple Linear Regression model

In [ ]:
cal_cv(LinearRegression(),'Simple Linear Regression',proba=False)

In [ ]:
# clist = []
# auclist = []

# for C in np.logspace(3,4,9):
#     label, auc, proba_auc = cal_cv(make_pipeline(PolynomialFeatures(2,include_bias=False),
#                                                 LogisticRegression(dual=False,
#                                                                    C=C,
#                                                                   class_weight='balanced',
#                                                                   max_iter=1500,
#                                                                   random_state=1,
#                                                                   solver='newton-cholesky')),
#                                   f'Poly Logistic Regression {C = }',
#                                   stand=True,
#                                   proba= True)
#     print(label,auc,proba_auc)
#     clist.append(C)
#     auclist.append(proba_auc)

# print(f'best C: {clist[auclist.index(max(auclist))]} and best AUC score: {max(auclist)}')

In [ ]:
# After hours of Computation Finally reached maxima of C which is C=1333.521432 

#  {'Simple Linear Regression': 'NOT PREDICTED',
#   'Poly Logistic Regression C = 0.01': 0.8472420399675084,
#   'Poly Logistic Regression C = 0.023713737056616554': 0.8472766265079873,
#   'Poly Logistic Regression C = 0.05623413251903491': 0.8472900599918038,
#   'Poly Logistic Regression C = 0.1333521432163324': 0.8472947535016143,
#   'Poly Logistic Regression C = 0.31622776601683794': 0.8472963534220834,
#   'Poly Logistic Regression C = 0.7498942093324559': 0.847297083040709,
#   'Poly Logistic Regression C = 1.7782794100389228': 0.8472975437242101,
#   'Poly Logistic Regression C = 4.216965034285822': 0.847297703207006,
#   'Poly Logistic Regression C = 10.0': 0.8472977647075984,
#   'Poly Logistic Regression C = 100.0': 0.8472978918749228,
#   'Poly Logistic Regression C = 133.3521432163324': 0.8472978970863222,
#   'Poly Logistic Regression C = 177.82794100389228': 0.8472978939601484,
#   'Poly Logistic Regression C = 237.13737056616552': 0.8472978908335108,
#   'Poly Logistic Regression C = 316.22776601683796': 0.8472978960456479,
#   'Poly Logistic Regression C = 421.6965034285823': 0.8472978939609769,
#   'Poly Logistic Regression C = 562.341325190349': 0.8472978950033415,
#   'Poly Logistic Regression C = 749.8942093324558': 0.8472978960453666,
#   'Poly Logistic Regression C = 1000.0': 0.8472978981302198,
#   'Poly Logistic Regression C = 1333.521432163324': 0.8472979033415864,       <----------
#   'Poly Logistic Regression C = 1778.2794100389228': 0.847297902299189,
#   'Poly Logistic Regression C = 2371.373705661655': 0.8472979022990648,
#   'Poly Logistic Regression C = 3162.2776601683795': 0.8472979033413381,
#   'Poly Logistic Regression C = 4216.965034285822': 0.8472979022991561,
#   'Poly Logistic Regression C = 5623.413251903491': 0.8472979022992803,
#   'Poly Logistic Regression C = 7498.942093324558': 0.8472978991722119,
#   'Poly Logistic Regression C = 10000.0': 0.8472978991722119})

In [ ]:
result_list = []
def cross_val(model, label):
    start_time = datetime.now()
    kf = StratifiedKFold(shuffle=True, random_state=1)
    oof = np.full(len(dff), np.nan)
    auc_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(x_temp, y_temp)):
        X_tr = x_temp.iloc[idx_tr]
        X_va = x_temp.iloc[idx_va]
        y_tr = y_temp.iloc[idx_tr]
        y_va = y_temp.iloc[idx_va]
        
        model.fit(X_tr, y_tr)
        try:
            y_va_pred = model.predict_proba(X_va)[:, 1]
        except AttributeError: # 'LinearSVC' object has no attribute 'predict_proba'
            y_va_pred = model.decision_function(X_va)
        oof[idx_va] = y_va_pred
        auc = roc_auc_score(y_va, y_va_pred)
        auc_list.append(auc)
    auc = np.array(auc_list).mean()
    execution_time = datetime.now() - start_time
    print(f"# AUC {auc:.15f}   time={str(execution_time)[-15:-7]}   {label}")
#     result_list.append((auc, label, execution_time))
    return auc

In [ ]:
# min_samples_leaf_list = []
# auclist = []

# for min_samples_leaf in [10,20,30,40,50,70, 80,90,100,150,170,200]:
#     start = datetime.now()
#     label,auc,proba_auc = cal_cv(make_pipeline(RandomForestClassifier(n_estimators=100, 
#                                                                     min_samples_leaf = min_samples_leaf,
#                                                                     max_features=1.0,
#                                                                     random_state=1)),
#                                 f'Random Forest with {min_samples_leaf = }',
#                                 stand=False,
#                                 proba=True)
#     print(label,auc,proba_auc)
#     print('Execution time:',datetime.now()-start)
#     print() 
#     min_samples_leaf_list.append(min_samples_leaf)
#     auclist.append(proba_auc)

In [ ]:
# for min_samples_leaf in [1,2,3,5,7,8,9,10]:
#     auc = cross_val(make_pipeline(
#                                   ExtraTreesClassifier(n_estimators=100,
#                                          min_samples_leaf=min_samples_leaf,
#                                          max_features=1.0,
#                                          random_state=1)),
#                     f"ET {min_samples_leaf=}")
    
# # At min_samples_leaf = 2 , AUC = 0.86644 

# # with select 1
# # AUC 0.811747948065179   time=0:03:13   ET min_samples_leaf=1
# # AUC 0.823468916440913   time=0:03:08   ET min_samples_leaf=2

# with select 2
# AUC 0.776322407240131   time=0:02:37   ET min_samples_leaf=1
# AUC 0.805048407534366   time=0:02:20   ET min_samples_leaf=2

In [ ]:
#* AUC 0.86079   time=0:04:50   ET min_samples_leaf=10
#* AUC 0.85709   time=0:04:11   ET min_samples_leaf=20
#* AUC 0.85243   time=0:03:39   ET min_samples_leaf=50
#* AUC 0.84863   time=0:03:28   ET min_samples_leaf=100
#* AUC 0.84618   time=0:03:49   ET min_samples_leaf=150
#* AUC 0.86552   time=0:07:09   ET min_samples_leaf=1
#* AUC 0.86644   time=0:06:07   ET min_samples_leaf=2
#* AUC 0.86615   time=0:05:56   ET min_samples_leaf=3
#* AUC 0.86449   time=0:05:18   ET min_samples_leaf=5
#* AUC 0.86287   time=0:05:18   ET min_samples_leaf=7
#* AUC 0.86210   time=0:05:28   ET min_samples_leaf=8
#* AUC 0.86169   time=0:05:22   ET min_samples_leaf=9
#* AUC 0.86079   time=0:05:30   ET min_samples_leaf=10

In [ ]:
# for n_neighbors in range(72, 77, 1):
#     auc = cross_val(make_pipeline(KNeighborsClassifier(n_neighbors=n_neighbors,
#                                                        weights='distance')),
#                     f"KNN {n_neighbors=}")
# # AUC 0.837221   time=0:01:14   KNN n_neighbors=73

In [ ]:
# # HistGradientBoostingClassifier
# auc = cross_val(HistGradientBoostingClassifier(random_state=1),
#                 f"HistGradientBoostingClassifier")
# # AUC 0.861615591771725   time=0:00:11   HistGradientBoostingClassifier

In [ ]:
# ensemble = VotingClassifier([('HistGrad',HistGradientBoostingClassifier(random_state=1)),
#                             ('RandForest',RandomForestClassifier(n_estimators=100,
#                                                                 min_samples_leaf = 2,
#                                                                 max_features=1.0,
#                                                                 random_state=1)),
#                             ('ExtraTree',ExtraTreesClassifier(n_estimators=100,
#                                                              min_samples_leaf =2,
#                                                              max_features=1.0,
#                                                              random_state=1))],
#                             voting='soft',
#                             weights=[0.2,0.4,0.4])

# auc = cross_val(ensemble,'HG + RF + ET')

In [ ]:
# cross_val(XGBClassifier(),'Simple XGB')
# cross_val(CatBoostClassifier(),'Simple CatBoost')
# cross_val(GradientBoostingClassifier(),'Simple GB')
# cross_val(LGBMClassifier(),'Simple LGBM')

# Simple XGB               0.864513
# Simple CatBoost          0.867679    
# Simple GB                0.854838
# Simple LGBM              0.861811

In [ ]:
# def objective(trial):
#     params = {
#         'n_estimators' : trial.suggest_int('n_estimators',500,1000),
#         "max_depth":trial.suggest_int('max_depth',3,50),
#         "learning_rate" : trial.suggest_float('learning_rate',1e-4, 0.25, log=True),
#         "min_child_weight" : trial.suggest_float('min_child_weight', 0.5,4),
#         "min_child_samples" : trial.suggest_int('min_child_samples',1,250),
#         "subsample" : trial.suggest_float('subsample', 0.2, 1),
#         "subsample_freq" : trial.suggest_int('subsample_freq',0,5),
#         "colsample_bytree" : trial.suggest_float('colsample_bytree',0.2,1),
#         'num_leaves' : trial.suggest_int('num_leaves', 2, 128),
#     }
#     lgbmmodel_optuna = LGBMClassifier(**params,random_state=1,device="gpu")
#     cv = cross_val_score(lgbmmodel_optuna, x_temp, y_temp, cv = 4,scoring='roc_auc').mean()
#     return cv

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100,timeout=2000)
# print(study.best_value) 

In [ ]:
# print(study.best_params)
# print()
# print(study.best_value)

In [ ]:
# best_param_yet = {'n_estimators': 919, 
#                   'max_depth': 19, 
#                   'learning_rate': 0.0487737902973404, 
#                   'min_child_weight': 1.558466463989505, 
#                   'min_child_samples': 216, 
#                   'subsample': 0.9462573827656944, 
#                   'subsample_freq': 5, 
#                   'colsample_bytree': 0.6329418580323135, 
#                   'num_leaves': 57}

# # For without ori
# # study.best_value: 0.8710607398384727
# new_param = {'n_estimators': 746, 'max_depth': 29, 
#              'learning_rate': 0.02889687137159223, 
#              'min_child_weight': 2.9072642727142783, 
#              'min_child_samples': 172, 
#              'subsample': 0.37716436254808705, 
#              'subsample_freq': 0, 
#              'colsample_bytree': 0.2858048478215879, 
#              'num_leaves': 122}

# label_cal,_,proba_auc,lgbm_df_one = cal_cv(LGBMClassifier(**new_param,device='gpu'), 'LGBM grid3',stand=False)
# print(label_cal, proba_auc) 
# # LGBM grid2 0.8662444218345303

# auc,lgbm_df_two = cross_val(LGBMClassifier(**new_param,device_type='gpu'),'LGBM grid3')

# # LGBM grid3 0.8695926989129232
# # # AUC 0.865065367700705   time=0:01:44   LGBM grid3

# # without ori
# # LGBM grid3 0.8717801385434415
# # # AUC 0.871034752354943   time=0:01:29   LGBM grid3

# # without ori and with new_param
# # LGBM grid3 0.8721356728366804
# # # AUC 0.871460319886812   time=0:01:24   LGBM grid3

In [ ]:
# def objective_xgb(trial):
#     params = {
#         'n_estimators' : trial.suggest_int('n_estimators',500,750),
#         'max_depth':  trial.suggest_int('max_depth',3,50),
#         'min_child_weight': trial.suggest_float('min_child_weight', 2,50),
#         "learning_rate" : trial.suggest_float('learning_rate',1e-4, 0.2,log=True),
#         'subsample': trial.suggest_float('subsample', 0.2, 1),
#         'gamma': trial.suggest_float("gamma", 1e-4, 1.0),
#         "colsample_bytree" : trial.suggest_float('colsample_bytree',0.2,1),
#         "colsample_bylevel" : trial.suggest_float('colsample_bylevel',0.2,1),
#         "colsample_bynode" : trial.suggest_float('colsample_bynode',0.2,1),
#     }
#     xgbmodel_optuna = XGBClassifier(**params,random_state=1,tree_method = "gpu_hist",eval_metric= "auc")   # verbose=500  not used
#     cv = cross_val_score(xgbmodel_optuna, x_temp, y_temp, cv = 4,scoring='roc_auc').mean()
#     return cv

# study_xgb = optuna.create_study(direction='maximize')
# study_xgb.optimize(objective_xgb, n_trials=100,timeout=5000)

# # Best is trial 91 with value: 0.8691347612295083.

In [ ]:
# print('Best params: ',study_xgb.best_params)
# print()
# print('Best Value: ',study_xgb.best_value)

# # {'n_estimators': 675, 
# #  'max_depth': 11, 
# #  'min_child_weight': 40.86869889025276, 
# #  'learning_rate': 0.05576041911952706, 
# #  'subsample': 0.9203427230250661, 
# #  'gamma': 0.1280337855575941, 
# #  'colsample_bytree': 0.8088703971777005, 
# #  'colsample_bylevel': 0.6225557766971302, 
# #  'colsample_bynode': 0.5064749820987031}

# # Best Value:  0.8691347612295083

In [ ]:

# xgb_optuna_param = {'n_estimators': 675, 
#      'max_depth': 11, 
#      'min_child_weight': 40.86869889025276, 
#      'learning_rate': 0.05576041911952706, 
#      'subsample': 0.9203427230250661, 
#      'gamma': 0.1280337855575941, 
#      'colsample_bytree': 0.8088703971777005, 
#      'colsample_bylevel': 0.6225557766971302, 
#      'colsample_bynode': 0.5064749820987031}

# label_cal,_,proba_auc = cal_cv(XGBClassifier(**xgb_optuna_param,tree_method='gpu_hist',eval_metric='auc',random_state=1), 
#                                'XGB Optuna Trial 1',stand=False)
# print(label_cal, proba_auc)

# auc = cross_val(XGBClassifier(**xgb_optuna_param,tree_method='gpu_hist',eval_metric="auc",random_state=1),'XGB Optuna Trial 1')
# # XGB Optuna Trial 1 0.8695700080964549
# # AUC 0.868799996766472   time=0:00:32   XGB Optuna Trial 1

## Out of Fold CROSS VALIDATION

In [ ]:
# SPLITS = 5
# REPEATS = 3
# lgb_scores = []
# xgb_scores = []
# cat_scores = []
# denominator=0

# for i, (tr_idx,val_idx) in enumerate(RepeatedStratifiedKFold(n_splits=SPLITS, n_repeats=REPEATS, random_state=1).split(x_temp,y_temp)):
#     x_tr,y_tr,x_val,y_val = x_temp.iloc[tr_idx],y_temp.iloc[tr_idx],x_temp.iloc[val_idx],y_temp.iloc[val_idx]
#     start = datetime.now()
#     print('-'*40,f'Fold {i+1}/{SPLITS*REPEATS} ','-'*40)
#     print('LGBM -->')
#     print()
    
#     lgbm = LGBMClassifier(**best_param_yet,random_state=1,device='gpu')
#     lgbm.fit(x_tr,y_tr, eval_set=[(x_val,y_val)], eval_metric='auc', early_stopping_rounds=500, verbose=500)
#     auc = roc_auc_score(y_val,lgbm.predict_proba(x_val)[:,1])
#     lgb_scores.append(auc)
#     denominator += 1
#     sub['smoking'] = sub['smoking'] + lgbm.predict_proba(test)[:,1]
#     print('-'*40,f'Fold {i+1} LGBM ROC score: {auc}    ','-'*40)
    
#     print()
#     print('CAT -->')
#     print()
    
#     cat = CatBoostClassifier(iterations=7500, verbose=500, od_type= 'Iter', eval_metric= 'AUC', random_state=1, 
#                              early_stopping_rounds = 500)
#     # eval_metric = 'auc' gives error in catboost
#     cat.fit(x_tr,y_tr, eval_set=[(x_val,y_val)], use_best_model=True)
#     auc = roc_auc_score(y_val,cat.predict_proba(x_val)[:,1])
#     cat_scores.append(auc)
#     denominator += 1
#     sub['smoking'] = sub['smoking'] + cat.predict_proba(test)[:,1]
#     print('-'*40,f'Fold {i+1} CATBOOST ROC score: {auc}   ','-'*40)
    
#     print()
#     print('XGB -->')
#     print()
    
#     xgb = XGBClassifier(**xgb_optuna_param, random_state=1, tree_method= 'gpu_hist', eval_metric='auc', early_stopping_rounds= 500)
#     xgb.fit(x_tr,y_tr, eval_set=[(x_val,y_val)], verbose=500, callbacks =[EarlyStopping(rounds=500, save_best=True)])
#     auc = roc_auc_score(y_val,xgb.predict_proba(x_val)[:,1])
#     xgb_scores.append(auc)
#     denominator += 1
#     sub['smoking'] += xgb.predict_proba(test)[:,1]
#     print('-'*40,f'Fold {i+1} XGBOOST ROC score: {auc  } ','-'*40)
    
#     print()
#     print('='*30,f'Execution time of Fold{i+1}: {datetime.now() - start}  ','='*30)
    
# print('\n\n','='*60)
# print(f'CV score of LGBM is {np.array(lgb_scores).mean()}')
# print(f'CV score of CatBoost is {np.array(cat_scores).mean()}')
# print(f'CV score of XGB is {np.array(xgb_scores).mean()}')

In [ ]:
# sub['smoking'] = sub['smoking']/denominator
# sub.to_csv('final_submission.csv',index=False)
# sub.head(10)